# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## The Data

** Importing the cal_housing_clean.csv file with pandas. Separating it into a training (70%) and testing set(30%).**

In [2]:
df = pd.read_csv("cal_housing_clean.csv")

In [4]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [6]:
df.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [7]:
from sklearn.model_selection import train_test_split

In [9]:
X = df.drop("medianHouseValue", axis = 1)

In [10]:
y = df["medianHouseValue"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [14]:
from sklearn.preprocessing import MinMaxScaler

In [15]:
scaler = MinMaxScaler()

In [16]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [18]:
X_train = pd.DataFrame(data = scaler.transform(X_train),columns = X_train.columns, index = X_train.index)

In [19]:
X_test = pd.DataFrame(data = scaler.transform(X_test), columns = X_test.columns, index = X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [22]:
X_train.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome'],
      dtype='object')

In [26]:
import tensorflow as tf

In [35]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
household = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [40]:
feat_cols = [age,rooms,bedrooms,population,household,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [41]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size=10, num_epochs=1000, shuffle = True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [42]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[7,7,7,7], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_model_dir': 'C:\\Users\\Jose\\AppData\\Local\\Temp\\tmp6dlnsqxd', '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [43]:
dnn_model.train(input_fn=input_func, steps = 1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Jose\AppData\Local\Temp\tmp6dlnsqxd\model.ckpt.
INFO:tensorflow:step = 1, loss = 4.71931e+11
INFO:tensorflow:global_step/sec: 197.739
INFO:tensorflow:step = 101, loss = 3.80495e+11 (0.503 sec)
INFO:tensorflow:global_step/sec: 277.583
INFO:tensorflow:step = 201, loss = 3.65555e+11 (0.365 sec)
INFO:tensorflow:global_step/sec: 275.344
INFO:tensorflow:step = 301, loss = 1.2881e+11 (0.363 sec)
INFO:tensorflow:global_step/sec: 236.011
INFO:tensorflow:step = 401, loss = 1.91757e+11 (0.424 sec)
INFO:tensorflow:global_step/sec: 259.97
INFO:tensorflow:step = 501, loss = 1.24334e+11 (0.383 sec)
INFO:tensorflow:global_step/sec: 292.653
INFO:tensorflow:step = 601, loss = 1.26925e+11 (0.347 sec)
INFO:tensorflow:global_step/sec: 243.877
INFO:tensorflow:step = 701, loss = 1.45775e+11 (0.409 sec)
INFO:tensorflow:global_step/sec: 273.143
INFO:tensorflow:step = 801, loss = 9.58281e+10 (0.362 sec)
INFO:tenso

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [62]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size = 10, num_epochs=1, shuffle=False)

In [63]:
pred_gen = dnn_model.predict(predict_input_func)

In [64]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\Jose\AppData\Local\Temp\tmp6dlnsqxd\model.ckpt-1000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [65]:
from sklearn.metrics import mean_squared_error

In [66]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [67]:
mean_squared_error(y_test,final_preds)**0.5

105174.46528373611